## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов.

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор.

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import nltk
nltk.download('stopwords')
import logging
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import re
from string import printable

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from nltk.stem import SnowballStemmer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.pipeline import Pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/MMMO/Data/sms_spam.csv')
data

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...
...,...,...
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."


In [ ]:
print(data.shape)
print(data[data['type'] == 'ham'].shape[0], data[data['type'] == 'spam'].shape[0])

(5559, 2)
4812 747


In [ ]:
stemmer = SnowballStemmer(language = 'english')

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;-]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-zа-я ]')
STOPWORDS = set(stopwords.words('english'))


def clean_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text).split()

    for i in range(len(text)):
      text[i] = stemmer.stem(text[i])

    text = ' '.join(word for word in text)

    return text

In [ ]:
data['text'] = data['text'].apply(clean_text)

In [ ]:
data

,type,text
0,ham,hope you are have a good week just check in
1,ham,kgive back my thank
2,ham,am also do in cbe onli but have to pay
3,spam,complimentari 4 star ibiza holiday or 10 000 c...
4,spam,okmail dear dave this is your final notic to c...
...,...,...
5554,ham,you are a great role model you are give so muc...
5555,ham,awesom i rememb the last time we got somebodi ...
5556,spam,if you dont your prize will go to anoth custom...
5557,spam,sms ac jsco energi is high but u may not know ...


In [ ]:
X = data['text']
y = data['type']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
clf = MultinomialNB()

In [ ]:
# https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

In [ ]:
cv_pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("clf", clf),
    ]
)

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 2), (1, 3)),  # unigrams or bigrams
    }

grid_search = GridSearchCV(cv_pipeline, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


GridSearchCV(estimator=Pipeline(steps=[('vect', CountVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__max_features': (None, 5000, 10000, 50000),
                         'vect__ngram_range': ((1, 2), (1, 3))},
             verbose=1)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters1 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters1[param_name]))

Best score: 0.985
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (2, 4)


In [ ]:
ngram = Pipeline(
    [
        ("vect", CountVectorizer(max_df = 0.5, max_features = None, ngram_range = (1, 2))),
        ("clf", clf),
    ]
)

ngram.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(max_df=0.5, ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [ ]:
n_pred = ngram.predict(X_test)

In [ ]:
tf_pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("clf", clf),
    ]
)

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__min_df': (0.0, 0.5, 0.75, 1.0),
    "vect__ngram_range": ((1, 2), (1, 3)),  # unigrams or bigrams
    }

grid_search = GridSearchCV(tf_pipeline, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
360 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
240 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py", line 390, in fit
    Xt = self._fit(X, y, **fit_params_steps)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/pipeline.py", line 348, in _fit
    X, fitted_transformer = fit_transform_one_cached(
  File "/usr/local/lib/python3.8/dist-packa

GridSearchCV(estimator=Pipeline(steps=[('vect', TfidfVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'vect__max_df': (0.5, 0.75, 1.0),
                         'vect__max_features': (None, 5000, 10000, 50000),
                         'vect__min_df': (0.0, 0.5, 0.75, 1.0),
                         'vect__ngram_range': ((1, 2), (1, 3))},
             verbose=1)

In [ ]:
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters2 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters2[param_name]))

Best score: 0.964
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: 5000
	vect__min_df: 0.0
	vect__ngram_range: (1, 2)


In [ ]:
tfidf_vec = pipeline2 = Pipeline(
    [
        ("vect", TfidfVectorizer(max_df = 0.5, max_features = 5000, min_df = 0.0, ngram_range = (1, 2))),
        ("clf", clf),
    ]
)

tfidf_vec.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.5, max_features=5000, min_df=0.0,
                                 ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

In [ ]:
tf_pred = tfidf_vec.predict(X_test)

In [ ]:
cv_char_pipeline = Pipeline(
    [
        ("vect", CountVectorizer(analyzer = 'char')),
        ("clf", clf),
    ]
)

parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 2), (2, 4), (5, 6)),  # unigrams or bigrams
    }

grid_search = GridSearchCV(cv_char_pipeline, parameters, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters3 = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters3[param_name]))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Best score: 0.985
Best parameters set:
	vect__max_df: 0.5
	vect__max_features: None
	vect__ngram_range: (2, 4)


In [ ]:
chargrams = Pipeline(
    [
        ("vect", CountVectorizer(analyzer = 'char', max_df = 1.0, max_features = None, ngram_range = (2, 4))),
        ("clf", clf),
    ]
)

chargrams.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', ngram_range=(2, 4))),
                ('clf', MultinomialNB())])

In [ ]:
ch_pred = chargrams.predict(X_test)

In [ ]:
dic = {
    'precision': [(precision_score(y_test, n_pred, average = 'macro')), (precision_score(y_test, tf_pred, average = 'macro')), (precision_score(y_test, ch_pred, average = 'macro'))],
    'recall': [(recall_score(y_test, n_pred, average = 'macro')), (recall_score(y_test, tf_pred, average = 'macro')), (recall_score(y_test, ch_pred, average = 'macro'))],
    'f1-score': [(f1_score(y_test, n_pred, average = 'macro')), (f1_score(y_test, tf_pred, average = 'macro')), (f1_score(y_test, ch_pred, average = 'macro'))],
    'accuracy': [(accuracy_score(y_test, n_pred)), (accuracy_score(y_test, tf_pred)), (accuracy_score(y_test, ch_pred))]
}

pd.DataFrame(data = dic, index = [['Мешок n-грамм', 'TFidf', 'Символьные n-граммы']])

,precision,recall,f1-score,accuracy
Мешок n-грамм,0.987368,0.949735,0.967463,0.985012
TFidf,0.979074,0.883273,0.923688,0.967026
Символьные n-граммы,0.975758,0.958421,0.966855,0.984412


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах.

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb')
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb')
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**:
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc')
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
re.findall(r'\b\w{1,2}', 'эх вот бы не было *****')

['эх', 'во', 'бы', 'не', 'бы']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie')
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2)
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
string = 'Преложения. Любые предложения. Абсолютно любые. Их много. Явно больше трех. Заметно больше. Помогите. Мне явно требуется психологическая помощь....'
re.split(r'[.]\s', string, maxsplit=2)

['Преложения',
 'Любые предложения',
 'Абсолютно любые. Их много. Явно больше трех. Заметно больше. Помогите. Мне явно требуется психологическая помощь....']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
string = '1 сказал *** *****, а уже в тюрьме на 10 лет'
re.sub('\d+', 'DIG', string)

'DIG сказал *** *****, а уже в тюрьме на DIG лет'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
string = 'https://isu.ifmo.ru/ Их сервера точно будут работать. Чьи? https://isu.ifmo.ru/'
re.sub(r'http\S+.', '', string)

'Их сервера точно будут работать. Чьи? '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog2 = re.compile('\w{2}[А-Яа-яё\-]+')
prog2.findall("Слова? Да, больше, ещё больше слов! Что-то ещё. Ко р откие сл ова.")

['Слова', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё', 'откие', 'ова']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
domens = re.compile(r'@[\w-]+\.[\w.-]+')
domens.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']